# <center> Web Scrapping of the website www.B17.ru </center>

For midterm project, we have to collect a raw data from the website www.b17.ru which contains numerous psychologists' profiles that we need to scrap onto our file. Here is the code that stands for parsing all profiles with respective attributes such as city,price,demand and so on. This data will help us to conduct more analysis such as Multiple Regression

In [2]:
from requests import get 
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import re
from tqdm import tqdm_notebook

In [3]:
## Function for city
def getCity(soup):
    try:
        q=soup.find('div',{'class':'status-grey'}).text
        city=re.findall('\г\. \w+\-\w+',q)[0]
        print(city)
    except: 
        q=soup.find('div',{'class':'status-grey'}).text
        city=re.findall('\г\. \w+',q)[0]

    return city    

In [4]:
## Function for price

def getPrice(soup):
    obj=soup.find('div',{'style':'margin: 10px 0 0; padding-left: 21px; position: relative;'})
    obj=obj.text
    x=re.search('Видео-консультация',obj)
    if (x!=None):
        return [str(s) for s in obj.split() if s.isdigit()][0]
    else:
        return np.nan

In [5]:
## Function for demand - doesnt work
def getProperties(soup,stats):
    try:
        obj= soup.find('div',{'style':'margin: 6px 0; font-size: 13px; text-align: center;'})
        qwe=obj.find('a',{'class':stats})
        qwe=qwe.text
        obj=[int(s) for s in qwe.split() if s.isdigit()]
    except: 
        obj=np.nan
    return obj

In [6]:
## Function for N publications
def publication(soup):
    try:
        pub=soup.find('div',{'class':'vkladki5 no-print'}).find('a',{'id':'vkl_article'}).find('span',{'class':'n'})
        pub=pub.text
    except:
        pub=0
    return pub  

In [7]:
def feedback(soup):
    try:
        fd=soup.find('div',{'class':'vkladki5 no-print'}).find('a',{'id':'vkl_otzyv'}).find('span',{'class':'n'})
        fd=fd.text
    except:
        fd=0
    return fd  

In [8]:
## Function for methods of work
def methods_work(soup):
    try:
        methods=soup.find('div',{'style':'margin: 20px 0;'}).text
        methods=re.split(r'\W+',methods)[3:]
        methods=' '.join(methods) #to convert from list to string
    except: methods=np.nan    
    return methods

In [9]:
## Function for marital status
def marital_status(soup):
    try:
        marital=soup.find_all('div',{'style':'margin: 10px 0;'})[2].text #several info with the same tags, parsed only MS (2)
        marital=re.split(r'\W+',marital)[2:]
        marital=' '.join(marital)
    except: marital=np.nan
    return marital

In [10]:
## Function for age
def age_con(soup):
    try:
        q=soup.find_all('div',{'style':'margin: 10px 0;'})[1].text
        q=re.findall('\d\d\d\d',q)
        q=''.join(q)
        q=2021-int(q)
    except: q=0    
    return q

In [11]:
## Function for education
def education(soup):
    edu=soup.find('div',{'id':'vuz'}).find('div',{'class':'vkladki_body_in vkladki_body_list'}).find('div',{'id':'vuz_list_s'}).text
    edu=list(edu.split("\n"))[:-1]
    return edu

### We finished to create a separate functions that retrieve a specific information from each profile. Lets combine it into single function

In [12]:
def psych(sor):
    
    #Created html paper

    response = get(sor, headers={'User-Agent': UserAgent().chrome})
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    
    if not response.ok:
        # если сервер нам отказал, вернем статус ошибки 
        return response.status_code
    
    #Apply function for demand
    demand=soup.find('div',{'style':'margin: 6px 0; font-size: 13px; text-align: center;'}).find('a')
    demand=demand.text
    demand=re.findall("\d+",demand)[0]

    
    #Apply function for price
    price=getPrice(soup)
    
    #Extract city 
    city=getCity(soup)
    
    #Extract name
    name=soup.find('h1').text
    
    #Extract number of Publications 
    pub=publication(soup)
    
    #Extract number of Feedbacks
    otz=feedback(soup)
    
    #Extract description
    des= soup.find('div',{'class':'vkladki_body0'})
    des=des.find('p')
    des=des.text
    
    #Extract methods of work
    methods=methods_work(soup)
    
    #Extract marital status
    marital=marital_status(soup)
    
    #Extract age
    age=age_con(soup)
    
    #Extract education
    edu=education(soup)
    data_col={
        "name":name,
        "demand":demand,
        "price":price,
        "publications":pub,
        "otzivy":otz,
        "description":des,
        "city":city,
        "methods":methods,
        "marital_status":marital,
        "age":age,
        "education":edu
    }
    
    return data_col

In [14]:
## Creating a dataset of single observation
df=pd.DataFrame(columns=["name","demand","price","publications","otzivy","description","city","methods","age","education"])
data_col=psych('https://www.b17.ru/ivanivanovichkuzmin/')
df=df.append(data_col,ignore_index=True)
df

г. Вольно-Надеждинское


,name,demand,price,publications,otzivy,description,city,methods,age,education,marital_status
0,Иван Кузьмин,690,NaN,161,57,Личный сайт: psy-ru-world.ruИнтернет-магазин (...,г. Вольно-Надеждинское,,44,[Саратовский военно-медицинский институт МО РФ...,Женат трое детей 1 внук Жена детский психолог ...


### Here is the first attemp to parse a single profile, so it seems succesfull. Now lets create a cycle that would retrieve all info from all profiles

In [15]:
#Page cycle
def pagelink(meme_page1):
    page_link='https://www.b17.ru/online/?page={}'.format(meme_page1)
    response1 = get(page_link, headers={'User-Agent': UserAgent().chrome})
    if not response1.ok:
        # если сервер нам отказал, вернем пустой лист для текущей страницы
        return [] 
    html1 = response1.content
    soup1 = BeautifulSoup(html1,'html.parser')
    x=soup1.find_all("a",{"class":"h"})[1:]
    lina=['https://www.b17.ru'+link.get("href") for link in x]
    return lina

### So far we are not going to parse all data from the website. Lets check our code on 5 pages of B17

In [16]:
#Retrieving n observations from website

df10=pd.DataFrame(columns=["name","demand","price","publications","otzivy","description","city","methods","age","education"])


for page in tqdm_notebook(range(5),desc='Pages'):
    links=pagelink(page)
    for links in tqdm_notebook(links,desc='Psy',leave=False):
        for i in range(5):
            try:
                data_col=psych(links)
                df10=df10.append(data_col,ignore_index=True)
                break
            except:
                
                continue
        time.sleep(3)        


<ipython-input-16-cd15fe68b7a4>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for page in tqdm_notebook(range(5),desc='Pages'):


<ipython-input-16-cd15fe68b7a4>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for links in tqdm_notebook(links,desc='Psy',leave=False):


г. Санкт-Петербург
г. Санкт-Петербург


г. Санкт-Петербург
г. Санкт-Петербург


г. Вольно-Надеждинское


г. Франкфурт-на
г. Санкт-Петербург



In [17]:
df10.head()

,name,demand,price,publications,otzivy,description,city,methods,age,education,marital_status
0,Кузьмичев Александр Сергеевич,967,7000,1453,175,Моя специализация – это эмоциональные расстрой...,г. Краснодар,Коучинг Поведенческая психотерапия Развитие Эм...,0,[Сибирский Государственный Медицинский Универс...,женат
1,Александр Молярук,543,4500,979,148,БЕСПЛАТНЫЙ онлайн-интенсив СТОП ТРЕВОГА: [ссыл...,г. Владивосток,Гештальт терапия Гипнотерапия Когнитивная тера...,51,"[МГУ им. адм. Невельского (Владивосток), Уссур...",Женат
2,Машин Владислав Леонидович,1076,2500,1483,93,"тел.: +7 ( 965 ) 317 - 56 - 12 адреса: м.""КУЗЬ...",г. Москва,Гештальт терапия Системная семейная терапия,43,"[Московский Социально-Педагогический Институт,...",в официальных отношениях не состою
3,Олег Владимирович Сурков,1010,2500,856,114,"Здравствуйте, я Олег Сурков. Мои консультации ...",г. Москва,Арт терапия Коучинг Поведенческая психотерапия...,63,"[МГУ им. М.В. Ломоносова, Образовательная прог...",есть
4,Наталия Филимонова,1494,4000,1487,55,"С парами тоже работаю online - одновременно, ж...",г. Санкт-Петербург,Арт терапия Гипнотерапия Когнитивно поведенчес...,53,"[Московский Государственный Университет, Ставр...",Замужем


In [102]:
df10.to_csv('data1.csv',index=False,encoding='UTF-8')